# COATI NLP model for encoding molecules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf, open_dict

from src.coati.models.io import load_e3gnn_smiles_clip_e2e
from src.modules.collate_fn import default_collate
from src.modules.losses import InfoNCE
from src.modules.molecules.coati import COATI

In [3]:
for i in range(1, 4):
    if not Path(f"../cpjump{i}/jump/").exists():
        print(f"Mounting cpjump{i}...")
        os.system(f"sshfs bioclust:/projects/cpjump{i}/ ../cpjump{i}")
    else:
        print(f"cpjump{i} already mounted.")

cpjump1 already mounted.
cpjump2 already mounted.
cpjump3 already mounted.


## Developping the model

In [2]:
smiles = [
    "CC1CC2=CCOC2O1",
    "OC1CC1(O)CC1CC1",
    "CC1N2C=NCC12C#C",
    "CC1COC11C(O)C1O",
    "CC12OCC(CO1)C2=O",
    "CC12CC(CO1)CC2=O",
    "CCN=COC",
    "CC1(CO)CO1",
    "C(C#N)C(=O)N",
    "CC(=O)OC=N",
]

In [60]:
model = COATI(
    pretrained_name="grande_closed",
    out_dim=128,
    padding_length=250,
    freeze=False,
    device="cuda",
)

Loading model from s3://terray-public/models/grande_closed.pkl
Loading tokenizer may_closedparen from s3://terray-public/models/grande_closed.pkl
number of parameters: 12.64M
number of parameters Total: 2.44M xformer: 17.92M Total: 20.36M 
vocab_name not found in tokenizer_vocabs, trying to load from file


In [63]:
model2 = model.to("cpu")

In [64]:
model2.device

'cpu'

In [66]:
model(smiles).shape

KeyboardInterrupt: 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
encoder, tokenizer = load_e3gnn_smiles_clip_e2e(
    # model parameters to load.
    doc_url="s3://terray-public/models/grande_closed.pkl",
    freeze=False,
    # device=torch.device("cpu"),
    # print_debug=True,
)

Loading model from s3://terray-public/models/grande_closed.pkl
Loading tokenizer may_closedparen from s3://terray-public/models/grande_closed.pkl
number of parameters: 12.64M
number of parameters Total: 2.44M xformer: 17.92M Total: 20.36M 
vocab_name not found in tokenizer_vocabs, trying to load from file


In [48]:
tokenizer.vocab

{'[PAD]': 0,
 '[STOP]': 1,
 '[SMILES]': 2,
 '[MASK]': 3,
 '[PREFIX]': 4,
 '[SUFFIX]': 5,
 '[MIDDLE]': 6,
 '[UNK]': 7,
 '[CLIP]': 8,
 '[FORMULA]': 9,
 '[GRAPH]': 10,
 '[EDGES]': 11,
 '[EDGE1]': 12,
 '[EDGEC]': 13,
 '[EDGE2]': 14,
 '[EDGE3]': 15,
 '[SET]': 16,
 '[ISOMORPHIC]': 17,
 '[VALID]': 18,
 '[TRUE]': 19,
 '[FALSE]': 20,
 '[geom_drugs]': 21,
 '[mcule]': 22,
 '[tspace_real]': 23,
 '[tensormol]': 24,
 '[chembl_mols]': 25,
 '[bbspace]': 26,
 '[zinc22]': 27,
 '[tspace_enum]': 28,
 '[ELM1]': 29,
 '[ELM2]': 30,
 '[ELM3]': 31,
 '[ELM4]': 32,
 '[ELM5]': 33,
 '[ELM6]': 34,
 '[ELM7]': 35,
 '[ELM8]': 36,
 '[ELM9]': 37,
 '[ELM10]': 38,
 '[ELM11]': 39,
 '[ELM12]': 40,
 '[ELM13]': 41,
 '[ELM14]': 42,
 '[ELM15]': 43,
 '[ELM16]': 44,
 '[ELM17]': 45,
 '[ELM18]': 46,
 '[ELM19]': 47,
 '[ELM20]': 48,
 '[ELM21]': 49,
 '[ELM22]': 50,
 '[ELM23]': 51,
 '[ELM24]': 52,
 '[ELM25]': 53,
 '[ELM26]': 54,
 '[ELM27]': 55,
 '[ELM28]': 56,
 '[ELM29]': 57,
 '[ELM30]': 58,
 '[ELM31]': 59,
 '[ELM32]': 60,
 '[ELM33]': 

In [47]:
tokenizer.__dir__()

['n_seq',
 'special_tokens',
 'smiles_tokens',
 'keys',
 'n_token',
 'vocab',
 'stop_token',
 'pad_token',
 'clip_token',
 'unk_token',
 'smiles_token',
 'suffix_token',
 'middle_token',
 'graph_token',
 'formula_token',
 'set_token',
 'smiles_trie',
 'special_trie',
 '__module__',
 '__doc__',
 '__init__',
 'pre_tokenize',
 'tokenize_text',
 'batch_smiles',
 'decode',
 '__dict__',
 '__weakref__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [28]:
encoder.xformer.transformer.ln_f.__dir__()

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook

In [39]:
batch_tokens = torch.tensor(
    [
        tokenizer.tokenize_text("[SMILES]" + s + "[STOP]", pad=True)
        if s != "*"
        else tokenizer.tokenize_text("[SMILES]C[STOP]", pad=True)
        for s in smiles
    ],
    device="cpu",
    dtype=torch.int,
)

In [40]:
batch_tokens.shape

torch.Size([10, 250])

In [38]:
tokenizer.n_seq = 250

In [32]:
tokenizer.__dir__()

['n_seq',
 'special_tokens',
 'smiles_tokens',
 'keys',
 'n_token',
 'vocab',
 'stop_token',
 'pad_token',
 'clip_token',
 'unk_token',
 'smiles_token',
 'suffix_token',
 'middle_token',
 'graph_token',
 'formula_token',
 'set_token',
 'smiles_trie',
 'special_trie',
 '__module__',
 '__doc__',
 '__init__',
 'pre_tokenize',
 'tokenize_text',
 'batch_smiles',
 'decode',
 '__dict__',
 '__weakref__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [9]:
batch_embeds = encoder.encode_tokens(batch_tokens, tokenizer)

In [10]:
encoder.encode_tokens?

Signature: encoder.encode_tokens(token_indices: torch.Tensor, tokenizer) -> torch.Tensor
Docstring: Embeds the tokens, and projects into the latent space.
File:      /mnt/2547d4d7-6732-4154-b0e1-17b0c1e0c565/Document-2/Projet2/Stage/workspace/jump_models/src/coati/models/encoding/clip_e2e.py
Type:      method

In [56]:
encoder.to("cpu")

e3gnn_smiles_clip_e2e(
  (point_encoder): e3gnn_clip(
    (act_fn): SiLU()
    (embedding): Linear(in_features=28, out_features=256, bias=True)
    (embedding_norm): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (node_dec): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): SiLU()
      (2): Identity()
      (3): Linear(in_features=256, out_features=256, bias=True)
    )
    (gcl_0): e_gcl_sparse(
      (instance_norm): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (edge_mlp): Sequential(
        (0): Linear(in_features=513, out_features=256, bias=True)
        (1): SiLU()
        (2): Identity()
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): SiLU()
        (5): Identity()
      )
      (node_mlp): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): SiLU()
        (2): Identity()
        (3):

In [57]:
encoder.batch_smiles_to_s2s_likelihood(smiles, tokenizer)

(tensor([ 5.1528,  3.8842, 41.5446,  4.8210,  7.4189,  2.2844,  2.4983,  0.7494,
         21.5638,  5.3194], grad_fn=<SumBackward1>),
 tensor([True, True, True, True, True, True, True, True, True, True]))

In [63]:
encoder.xformer.forward(batch_tokens)

tensor([[[ -4.7370,   0.3078,   1.6261,  ...,  -0.0516,  -3.1638,   0.5547],
         [-11.4987,  -0.3192,  -1.5464,  ...,  -6.1698, -10.2218,  -3.1609],
         [ -8.4435,   0.8021,  -0.8089,  ...,  -2.4167,  -7.5098,  -2.4759],
         ...,
         [-10.3736,   2.5105,  -0.7439,  ...,  -4.9863,  -8.6617,  -3.3883],
         [-10.2333,   2.5897,  -0.6997,  ...,  -4.8988,  -8.5454,  -3.3025],
         [-10.0865,   2.5194,  -0.6686,  ...,  -4.8681,  -8.4047,  -3.2709]],

        [[ -4.7370,   0.3078,   1.6261,  ...,  -0.0516,  -3.1638,   0.5547],
         [-11.6163,  -2.5803,  -1.9344,  ...,  -5.1848, -10.3400,  -5.3778],
         [ -9.6973,   3.3149,  -1.9919,  ...,  -2.9876,  -7.5202,  -5.2770],
         ...,
         [-10.5352,   4.5065,  -0.2985,  ...,  -4.9456,  -8.7423,  -3.3941],
         [-10.7862,   4.4251,  -0.2186,  ...,  -5.1446,  -8.9914,  -3.7423],
         [-10.8848,   4.3139,  -0.1930,  ...,  -5.3303,  -9.0748,  -3.8258]],

        [[ -4.7370,   0.3078,   1.6261,  ...

## Check config

In [4]:
# GlobalHydra.instance().clear()

In [5]:
initialize(version_base=None, config_path="../configs")

hydra.initialize()

In [17]:
cfg = compose(
    config_name="train.yaml",
    overrides=[
        "evaluate=true",
        "eval=hint",
        "paths.projects_dir=..",
        "paths.output_dir=./tmp/21312FS12A",
        "trainer.devices=1",
        "seed=22123",
        "experiment=coati/med",
        "trainer=gpu",
        "trainer.devices=[1]",
        "trainer.max_epochs=200",
        "data.num_workers=12",
        "data.transform.size=224",
        "data.batch_size=4",
        "model.embedding_dim=256",
        "model/image_encoder=vit_base_16_224",
        "model/criterion=ntxent_reg",
        "model.criterion.alpha=0.2",
        "model.criterion.mse_reg=0.5",
        "model.criterion.variance_reg=1",
        "model.criterion.covariance_reg=0.25",
        "model.criterion.temperature=10",
        "model.criterion.temperature_requires_grad=True",
    ],
)
print(OmegaConf.to_yaml(cfg))

task_name: train
tags:
- med_jump_cl
- nlp_coati
- clip_like
- ${model.molecule_encoder.pretrained_name}
- ${model.image_encoder.instance_model_name}
train: true
test: true
evaluate: true
compile: false
ckpt_path: null
seed: 22123
data:
  compound_transform:
    _target_: src.modules.compound_transforms.coati.COATITransform
    compound_str_type: inchi
    pretrained_name: ${model.molecule_encoder.pretrained_name}
    padding_length: ${model.molecule_encoder.padding_length}
    model_dir: ${paths.model_dir}
  _target_: src.models.jump_cl.datamodule.BasicJUMPDataModule
  batch_size: 4
  num_workers: 12
  pin_memory: null
  prefetch_factor: 3
  drop_last: true
  transform:
    _target_: src.modules.transforms.DefaultJUMPTransform
    _convert_: object
    size: 224
    dim:
    - -2
    - -1
  force_split: false
  splitter:
    _target_: src.splitters.ScaffoldSplitter
    train: 24576
    test: 3072
    val: 2048
    retrieval: 0
  use_compond_cache: false
  data_root_dir: ${paths.projec

In [18]:
dm = instantiate(cfg.data)

vocab_name not found in tokenizer_vocabs, trying to load from file


In [19]:
dm.prepare_data()
dm.setup("fit")

In [20]:
dl = dm.train_dataloader()

In [21]:
b = next(iter(dl))

In [22]:
b["compound"].shape

torch.Size([4, 250])

In [23]:
b["image"].shape

torch.Size([4, 5, 224, 224])

In [24]:
model = instantiate(cfg.model)

vocab_name not found in tokenizer_vocabs, trying to load from file
number of parameters: 12.64M
number of parameters Total: 2.44M xformer: 17.92M Total: 20.36M 


In [25]:
bb = {k: v.to("cuda") if isinstance(v, torch.Tensor) else v for k, v in b.items()}
model = model.to("cuda")

In [26]:
embs = model.forward(**bb)

In [27]:
embs["compound_emb"].shape

torch.Size([4, 256])

In [28]:
embs["image_emb"].shape

torch.Size([4, 256])

In [29]:
info_nce = InfoNCE(temperature=1, return_rank=True, norm=True, eps=0)

In [30]:
z1 = embs["compound_emb"]
z2 = embs["image_emb"]

In [31]:
info_nce(z1, z2)

{'loss': tensor(1.3974, device='cuda:0', grad_fn=<NegBackward0>),
 'x_to_y_top1': tensor(0., device='cuda:0'),
 'x_to_y_top5': tensor(1., device='cuda:0'),
 'x_to_y_top10': tensor(1., device='cuda:0'),
 'x_to_y_mean_pos': tensor(3.2500, device='cuda:0'),
 'x_to_y_mean_pos_normed': tensor(0.8125, device='cuda:0'),
 'y_to_x_top1': tensor(0., device='cuda:0'),
 'y_to_x_top5': tensor(1., device='cuda:0'),
 'y_to_x_top10': tensor(1., device='cuda:0'),
 'y_to_x_mean_pos': tensor(2.5000, device='cuda:0'),
 'y_to_x_mean_pos_normed': tensor(0.6250, device='cuda:0')}

In [31]:
info_nce(z2, z1)

{'loss': tensor(3.4728, device='cuda:0', grad_fn=<NegBackward0>),
 'x_to_y_top1': tensor(0., device='cuda:0'),
 'x_to_y_top5': tensor(0.1875, device='cuda:0'),
 'x_to_y_top10': tensor(0.3125, device='cuda:0'),
 'x_to_y_mean_pos': tensor(17.2500, device='cuda:0'),
 'y_to_x_top1': tensor(0.0312, device='cuda:0'),
 'y_to_x_top5': tensor(0.2500, device='cuda:0'),
 'y_to_x_mean_pos': tensor(17.8125, device='cuda:0')}